In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
path = "/content/drive/MyDrive/연구실/"

In [4]:
!pip install git+https://github.com/hyperopt/hyperopt.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/hyperopt/hyperopt.git to /tmp/pip-req-build-05c9zd18
  Running command git clone --filter=blob:none --quiet https://github.com/hyperopt/hyperopt.git /tmp/pip-req-build-05c9zd18
  Resolved https://github.com/hyperopt/hyperopt.git to commit 6c5b33be93a71a4ae6a38f04a29b18020b78b702
  Running command git submodule update --init --recursive -q
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 KB 6.1 MB/s eta 0:00:00
  Created wheel for hyperopt: filename=hyperopt-0.2.7-py2.py3-none-any.whl size=966138 sha256=e0c7ae4278670fdcbe634586d3c864a02ba2f662a73f92b7b175a57c37905b85
  Stored in directory: /tmp/pip-ephem-wheel-cache-t1lrfx63/wheels/7e/a2/bc/da25ffbedcf76e6957370dd98f8bfc0b9cda94cef6c350d117
Successfully built hyperopt
  Attempting uninstall: hyperopt
    Found existing installation: hyperopt 0.1.2
    Uninstallin

In [5]:
import pandas as pd
import numpy as np

from hyperopt import hp
from hyperopt import STATUS_OK
from hyperopt.early_stop import no_progress_loss

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from tqdm import tqdm
import joblib

import warnings
warnings.filterwarnings("ignore")

In [6]:
data = pd.read_csv(path+'전처리최종이었음좋겠다.csv')
data.columns

Index(['yearmonth', 'Ticker', '한국표준산업분류코드10차(중분류)', '수익률 (1개월)(%)',
       '베타 (D,1Yr)', 'PER(배)', 'PSR(배)', 'chmom', 'mom36m', 'zerotrade',
       'std_turn', 'mean_ill', 'retvol', 'maxret', 'var_market', 'mom1m',
       'mom12m', 'indmom', 'turn', 'mvel1', 'dolvol', 'betasq', 'Idiovol',
       'macro_(콜금리-기준금리)평균', 'macro_원/달러환율', 'macro_금($/OZ)', 'macro_뉴스심리지수',
       'macro_KOSPI dp', 'macro_KOSPI PER', 'macro_KOSPI PBR', 'macro_CD Rate',
       'macro_Term spread', 'macro_Default Spread', 'macro_VKOSPI 200'],
      dtype='object')

In [7]:
X = data.drop(['한국표준산업분류코드10차(중분류)', '수익률 (1개월)(%)', 'macro_뉴스심리지수'], axis=1)
y = data['수익률 (1개월)(%)']
ym = data['yearmonth']
tk = data['Ticker']

In [8]:
X.shape

(312185, 31)

In [9]:
a = '2004|2005|2006|2007|2008|2009|2010|2011|2012|2013|2014|2015|2016|2017|2018|2019|2020'
ya0 = []
tk0 = []
ym0 = []
ya1 = []

for i in tqdm(range(9)): 
    # tX = X[ym.str.contains(a[:24+5*i])]
    # vX = X[ym.str.contains(a[25+5*i:39+5*i])]
    tvX = X[ym.str.contains(a[:39+5*i])]
    tvy = ym[ym.str.contains(a[:39+5*i])]
    oX = X[ym.str.contains(a[40+5*i:44+5*i])]
    # ty = y[ym.str.contains(a[:24+5*i])]
    # vy = y[ym.str.contains(a[25+5*i:39+5*i])]
    tvy = y[ym.str.contains(a[:39+5*i])]
    oy = y[ym.str.contains(a[40+5*i:44+5*i])]
    otk = tk[ym.str.contains(a[40+5*i:44+5*i])]
    oym = ym[ym.str.contains(a[40+5*i:44+5*i])]
    
    # 하이퍼파라미터 검색 공간 설정
    pcr_search_space = {
        'n_components' : hp.quniform('n_components', 1, 29, 1)
    }
    
    # 목적 함수 정의
    def objective_func(search_space):
        pcr = Pipeline([
            ('scaler', StandardScaler()),
            ('PCR', PCA(n_components = int(search_space['n_components']))),
            ('LR', LinearRegression())
        ])
        
        r = 1
        
        # time series cv
        for j in range(3): # 마지막 3년만 valid로 사용
            tX = X[ym.str.contains(a[:24+5*(j+i)])]
            vX = X[ym.str.contains(a[25+5*(j+i):29+5*(j+i)])]
            ty = y[ym.str.contains(a[:24+5*(j+i)])]
            vy = y[ym.str.contains(a[25+5*(j+i):29+5*(j+i)])]
            
            # for문에 필요한 key를 미리 만듦
            ym_temp = vX['yearmonth'].reset_index(drop=True) # t = ym_temp==k를 위해 reset index
            key_yearmonth = dict.fromkeys(ym_temp)
            
            # vy의 index를 reset
            vy.reset_index(drop=True, inplace=True) # r = r*(vy[sort[-d:].index].mean()+100)/100를 위해 reset index
            
            # 년도, 티커 드랍
            tX = tX.drop(['yearmonth', 'Ticker'], axis=1)
            vX = vX.drop(['yearmonth', 'Ticker'], axis=1)
            
            # fitting
            pcr.fit(tX, ty)
            
            # 수익률 metric 계산           
            y_pred = pd.Series(pcr.predict(vX).reshape(-1, )) # sort_values() 사용을 위해 변환
    
            for k in list(key_yearmonth):
                t = ym_temp==k # 해당 month
                d = round(len(y_pred[t])/10)
                sort = y_pred[t].sort_values()
                r = r*(vy[sort[-d:].index].mean()+100)/100
                
        # 새로운 loss 정의
        new_loss = r*(-1)
        
        return {
            'loss':new_loss,
            'status':STATUS_OK
        }
    
    # fmin()을 사용하여 최적 하이퍼 파라미터 찾기
    from hyperopt import fmin, tpe, Trials
    import warnings
    warnings.filterwarnings('ignore')

    trial_val = Trials() # 결과 저장
    
    best = fmin(
        fn=objective_func,
        space=pcr_search_space,
        algo=tpe.suggest,
        max_evals=1000,    # 최대 반복 횟수 지정
        trials=trial_val,
        return_argmin=False, # hp.choice의 경우 값 대신 index를 반환함 
        early_stop_fn=no_progress_loss(20), # 30회 동안 개선되지 않으면 조기중지
        # https://github.com/hyperopt/hyperopt/issues/689
        rstate=np.random.default_rng(42) # https://github.com/hyperopt/hyperopt/issues/838
    )
    
    # 정수형 하이퍼 파라미터 값이 뒤에 .0이 붙어서 실수형으로 반환됨을 유의하기!
    best['n_components'] = int(best['n_components'])
    print(best) 
    
    reg0 = Pipeline([
            ('scaler', StandardScaler()),
            ('PCR', PCA(**best)),
            ('LR', LinearRegression())
        ])
    
    import warnings
    warnings.filterwarnings('ignore')
    
    tvX = tvX.drop(['yearmonth', 'Ticker'], axis=1)
    oX = oX.drop(['yearmonth', 'Ticker'], axis=1)
    
    reg0.fit(tvX, tvy)
    
    yy0 = reg0.predict(oX)
    ya0 = ya0 + oy.tolist()
    ya1 = ya1 + yy0.tolist()
    tk0 = tk0 + otk.tolist()
    ym0 = ym0 + oym.tolist()
    joblib.dump(reg0, path+'PCR_irr_'+str(i)+'.pkl')
    
PCR = pd.DataFrame({'real':ya0,'predict':ya1,'ticker':tk0,'yearmonth':ym0})
PCR.to_csv(path+'PCR_irr.csv')  

  0%|          | 0/9 [00:00<?, ?it/s]


  4%|▍         | 45/1000 [02:57<1:02:56,  3.95s/trial, best loss: -6.594752538976164]
{'n_components': 28}


 11%|█         | 1/9 [03:03<24:30, 183.81s/it]


  4%|▍         | 41/1000 [02:56<1:08:43,  4.30s/trial, best loss: -4.369182807282383]
{'n_components': 28}


 22%|██▏       | 2/9 [06:01<21:02, 180.43s/it]

...

 78%|███████▊  | 7/9 [23:51<07:21, 220.68s/it]


  4%|▍         | 41/1000 [04:53<1:54:28,  7.16s/trial, best loss: -2.4359106623722475]
{'n_components': 18}


 89%|████████▉ | 8/9 [28:49<04:05, 245.28s/it]


  4%|▍         | 41/1000 [05:15<2:02:54,  7.69s/trial, best loss: -1.917197958207744]
{'n_components': 18}


100%|██████████| 9/9 [34:08<00:00, 227.62s/it]


In [10]:
ya0 = pd.Series(PCR['real'])
ya1 = pd.Series(PCR['predict'])
tk0 = pd.Series(PCR['ticker'])
ym0 = pd.Series(PCR['yearmonth'])

r1 = []

for i in list(dict.fromkeys(ym0)):
    t = ym0==i
    d = round(len(ya1[t])/10)
    sort = ya1[t].sort_values()
    r1.append(ya0[sort[-d:].index].mean())

rr1 = 1
rpcr1 = [1]
for i in r1:
    rr1 = rr1*(i+100)/100
    rpcr1.append(rr1)

In [11]:
rpcr1[-1]

34.936168511674616